In [1]:
%%capture
!pip install flask
!pip install flask_cors
!pip install pyngrok

In [ ]:
# %%capture
# !pip install unsloth
# !pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
# !pip install vllm

In [2]:
%%capture
!pip install --no-deps unsloth vllm
# [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
# Skip restarting message in Colab
import sys, re, requests; modules = list(sys.modules.keys())
for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

# vLLM requirements - vLLM breaks Colab due to reinstalling numpy
f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
with open("vllm_requirements.txt", "wb") as file:
    file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
!pip install -r vllm_requirements.txt

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Lexical Bias Detection

In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# fine-tuned RoBERTa for detection
lexical_detection_checkpoint = '/content/drive/Shareddrives/FYP 2024-2025/Phase-2/Lexical Bias Detection/News/saved_models/RoBERTa_Weighted'
lexical_detection_tokenizer = AutoTokenizer.from_pretrained(lexical_detection_checkpoint, local_files_only=True)
lexical_detection_model = AutoModelForSequenceClassification.from_pretrained(lexical_detection_checkpoint, local_files_only=True)

lexical_detection_model.to("cuda")
lexical_detection_model.eval() # set in eval mode

tokenizer_config.json:   0%|          | 0.00/380 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [9]:
def run_detection(text):
  global lexical_detection_tokenizer, lexical_detection_model
  inputs = lexical_detection_tokenizer(text, max_length=256, return_tensors="pt", truncation=True, padding="max_length").to("cuda")
  with torch.no_grad():
    outputs = lexical_detection_model(**inputs)
  logits = outputs.logits
  predictions = torch.argmax(logits, dim=-1)
  del inputs, outputs, logits # manual memory freeing
  return predictions.item()

### Lexical Bias Mitigation

In [10]:
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template
max_seq_length = 2048
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True

# finetuned phi-3.5 for mitigation
lexical_mitigation_model, lexical_mitigation_tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/content/drive/Shareddrives/FYP 2024-2025/Phase-2/Lexical Bias Mitigation/News/models/phi-3.5-3.8b-inference",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

lexical_mitigation_tokenizer = get_chat_template(
    lexical_mitigation_tokenizer,
    chat_template = "phi-3",
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt", "system": "system"}, # ShareGPT style
)
# set model for inference
FastLanguageModel.for_inference(lexical_mitigation_model)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-05 08:10:17 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.3. vLLM: 0.8.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/140 [00:00<?, ?B/s]

Unsloth 2025.3.19 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.

In [11]:
def run_mitigation(text):
  global lexical_mitigation_tokenizer, lexical_mitigation_model
  messages = [
    {"from": "system", "value": "You are provided with a linguistically biased sentence from a news article. Linguistic bias refers to the use of words, phrases, or structures that carry a subjective tone, either overly positive or overly negative. Such biases can influence the reader's perception, making the text less neutral. Your task is to rephrase the given sentence to remove or neutralize the bias in such a way that most of its original meaning, content and structure is retained."},
    {"from": "human", "value": "The biased sentence is: " + text}
  ]
  inputs = lexical_mitigation_tokenizer.apply_chat_template(
      messages,
      tokenize = True,
      add_generation_prompt = True,
      return_tensors = "pt",
  ).to("cuda")
  outputs = lexical_mitigation_model.generate(input_ids = inputs, max_new_tokens = 256, use_cache = True)
  text_output = lexical_mitigation_tokenizer.batch_decode(outputs)[0]
  text_output = text_output.split("<|assistant|>")[1].strip()
  text_output = text_output.split("<|end|>")[0].strip()
  del inputs, outputs # manual memory freeing
  return text_output

### Neutral Roundup Generation

In [12]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
max_seq_length = 5000
lora_rank = 16

roundup_model, roundup_tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/content/drive/Shareddrives/FYP 2024-2025/Phase-2/SummaryGen/models/phi-3.5-grpo-1-epoch",
    max_seq_length = max_seq_length,
    dtype = None,
    load_in_4bit = True,
    fast_inference = True,
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.3,
)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.3. vLLM: 0.8.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/phi-3.5-mini-instruct-bnb-4bit with actual GPU utilization = 27.49%
Unsloth: Your GPU has CUDA compute capability 8.0 with VRAM = 39.56 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 5000. Num Sequences = 224.
Unsloth: vLLM's KV Cache can use up to 8.34 GB. Also swap space = 6 GB.
INFO 04-05 08:11:45 [config.py:585] This model supports multiple tasks: {'embed', 'score', 'generate', 'reward', 'classify'}. Defaulting to 'generate'.
WARNING 04-05 08:11:46 [arg_utils

tokenizer_config.json:   0%|          | 0.00/3.37k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

...

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 04-05 08:11:57 [punica_selector.py:18] Using PunicaWrapperGPU.
INFO 04-05 08:11:57 [model_runner.py:1146] Model loading took 2.2139 GB and 4.761090 seconds
INFO 04-05 08:12:06 [worker.py:267] Memory profiling takes 8.20 seconds
INFO 04-05 08:12:06 [worker.py:267] the current vLLM instance can use total_gpu_memory (39.56GiB) x gpu_memory_utilization (0.27) = 10.87GiB
INFO 04-05 08:12:06 [worker.py:267] model weights take 2.21GiB; non_torch_memory takes 0.02GiB; PyTorch activation peak memory takes 0.37GiB; the rest of the memory reserved for KV Cache is 8.27GiB.
INFO 04-05 08:12:06 [executor_base.py:111] # cuda blocks: 1411, # CPU blocks: 1024
INFO 04-05 08:12:06 [executor_base.py:116] Maximum concurrency for 5000 tokens per request: 4.52x
INFO 04-05 08:12:11 [model_runner.py:1442] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If ou

Capturing CUDA graph shapes: 100%|██████████| 31/31 [00:51<00:00,  1.66s/it]

INFO 04-05 08:13:02 [model_runner.py:1570] Graph capturing finished in 51 secs, took 0.78 GiB
INFO 04-05 08:13:02 [llm_engine.py:447] init engine (profile, create kv cache, warmup model) took 64.65 seconds



Not an error, but Unsloth cannot patch Attention layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Not an error, but Unsloth cannot patch O projection layer with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.3.19 patched 32 layers with 0 QKV layers, 0 O layers and 32 MLP layers.


In [13]:
from vllm import SamplingParams

def generate_roundup_summary_from_text(event: str, left_article: str, center_article: str, right_article: str, lora_name: str = "/content/drive/Shareddrives/FYP 2024-2025/Phase-2/SummaryGen/models/phi-3.5-grpo-1-epoch-lora") -> str:

    global roundup_model, roundup_tokenizer
    # Define system message
    system_message = (
        "You are given an event headline and news articles from the left, right, and center perspectives. "
        "Your task is to summarize the event in three to four sentences, and mention how different the different "
        "perspectives covered the event. To do this, you can highlight what each perspective emphasized on. "
        "This summary must be factual and neutral without any bias."
    )

    # Construct prompt in chat format
    prompt = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content':  f"### Event Headline\n{event}\n"
                                     f"### Left Article\n{left_article}\n"
                                     f"### Center Article\n{center_article}\n"
                                     f"### Right Article\n{right_article}"}
    ]

    # Format with tokenizer's chat template
    prompt_text = roundup_tokenizer.apply_chat_template(
        prompt,
        tokenize=False,
        add_generation_prompt=True
    )

    # Sampling settings
    sampling_params = SamplingParams(
        temperature=0.8,
        top_p=0.95,
        max_tokens=1024,
    )

    # Generate and return the summary
    output = roundup_model.fast_generate(
        prompt_text,
        sampling_params=sampling_params,
        lora_request=roundup_model.load_lora(lora_name),
    )[0].outputs[0].text

    return output


In [14]:
# Test
event = "Pressley To Push For Kavanaugh Impeachment Inquiry"
left_article = ["WASHINGTON — House Judiciary Committee Chairman Jerry Nadler, D-N.Y., responded to calls for an investigation into Supreme Court Justice Brett Kavanaugh in light of new sexual misconduct allegations by saying that the panel has \"our hands full with impeaching the president.\"", "In a radio interview with WNYC on Monday, Nadler was asked if he’d be concerned with Democrats thinking he’s not taking the Kavanaugh allegations seriously enough.", "He said his committee has too much on its plate.", "\"We have our hands full with impeaching the president right now and that’s going to take up our limited resources and time for a while,\" Nadler said.", "Asked about whether he plans to keep his promise from last October of investigating Kavanaugh over his Senate confirmation hearing, Nadler was candid about his position on impeaching the president.", "\"Frankly, we are concentrating our resources on determining whether to impeach the president. Personally I think the President ought to be impeached but we have to concentrate on that for the next few months,\" he said.", "Over the weekend, a pair of New York Times reporters in an opinion-section article previewing their upcoming book on Kavanaugh's confirmation said they found new corroboration for accusations that Kavanaugh exposed himself to Deborah Ramirez, a classmate at Yale.", "NBC News has not verified that reporting.", "The reporters said that former Yale classmate Max Stier told senators and the FBI about a different episode of alleged sexual misconduct.", "Two people with first-hand knowledge confirm to NBC News that the FBI was notified of Stier's claim.", "The chairman said that his committee plans to question FBI Director Christopher Wray next month including about the agency’s investigation into Kavanaugh.", "Nadler continued by saying he personally believes impeaching the president is \"imperative\" so that Democrats can \"vindicate the Constitution,\" acknowledging that its purpose would not be removing Trump from office because the GOP-controlled Senate wouldn’t do that.", "Nadler said that while he feels there is already enough information to impeach the President, he believes the Committee needs to educate the public to get behind impeachment before moving forward.", "\"We have to show that there are adequate grounds for impeachment, that there are imperative grounds for impeachment and convince people. If that happens, if people are convinced after the hearings that the president should be impeached then we will be able to get the votes, if that doesn’t happen, we won’t be able to get the votes,\" Nadler said.", "The Judiciary Committee approved a resolution last Thursday that set procedures and rules for future impeachment investigation hearings.", "The first such hearing will be held on Tuesday afternoon with Trump’s former campaign manager, Corey Lewandowski, as the first witness.", "Some top Democrats, including Sens. Kamala Harris, D-Calif. and Elizabeth Warren, D-Mass., called for Kavanaugh to be impeached in light of the new allegations against him.", "The revelations today confirm what we already knew: During his hearing, Kavanaugh faced credible accusations and likely lied to Congress. I support any appropriate constitutional mechanism to hold him accountable. — Bernie Sanders (@BernieSanders) September 15, 2019", "Kavanaugh was sworn in to the high court last October after senators investigated sexual assault and misconduct allegations made against him by Christine Blasey Ford and other women during his Senate confirmation process.", "He has denied the allegations made against him."]
right_article = ["Rep. Ayanna Pressley, D-Mass., one of the four freshman congresswomen known collectively as the “Squad,” is planning Tuesday to introduce a resolution seeking an impeachment inquiry into Supreme Court Associate Justice Brett Kavanaugh, despite reluctance from top Democrats to pursue one and concerns about the basis for the latest accusations.", "The resolution follows a New York Times report that included a new allegation of sexual misconduct from when Kavanaugh was in college -- although an editors' note later revealed that the alleged victim did not recall it taking place.", "In a statement, Pressley only focused on the women who came forward during Kavanaugh’s confirmation process.", "“I believe Christine Blasey Ford. I believe Deborah Ramirez,” Pressley said in a statement shared by Boston.com reporter Nik DeCosta-Klipa, which Pressley retweeted from her official Twitter account. “It is our responsibility to collectively affirm the dignity and humanity of survivors.”", "Pressley also used her personal Twitter account to retweet a post that included images of what appears to be text from the resolution.", "It calls for “Inquiring whether the House of Representatives should impeach Brett Michael Kavanaugh, Associate Justice for the Supreme Court of the United States of America,” and for the House Judiciary Committee to conduct depositions and take affidavits, serving subpoenas if necessary.", "Rep. Jerrold Nadler, D-N.Y., who heads the House Judiciary Committee, has not necessarily come out against impeaching Kavanaugh.", "But he made clear it was not a top priority for his committee.", "“We have our hands full with impeaching the president right now,” Nadler said on a radio show Monday.", "Top-ranking Sen. Dick Durbin, D-Ill., outright rejected the calls.", "“We’ve got to get beyond this ‘impeachment is the answer to every problem,” Durbin said, according to Politico. “It’s not realistic. If that’s how we are identified in Congress, as the impeachment Congress, we run the risk that people will feel we’re ignoring the issues that mean a lot to them as families.”", "GOP lawmakers, meanwhile, quickly rallied around Kavanaugh Monday and vowed to fight any impeachment effort.", "South Carolina Republican Sen. Lindsey Graham, the current Judiciary Committee chairman who mounted one of the most passionate defenses of Kavanaugh during his confirmation hearing, called the attacks “beyond the pale.”", "“My heart goes out to Justice Kavanaugh’s family for being forced to endure this ridiculous treatment once again,” he tweeted.", "The original Times piece, adapted from a forthcoming book, asserted that a Kavanaugh classmate, Clinton-connected nonprofit CEO Max Stier, “saw Mr. Kavanaugh with his pants down at a different drunken dorm party, where friends pushed his penis into the hand of a female student.”", "By late Sunday, the paper had revised its story to include the fact that several friends of the alleged victim said she told them she did not recall the reported sexual assault in question.", "Several Democratic presidential candidates, however, said that Kavanaugh should be impeached and did not back down after the editors' note and revision to the story.", "Sens. Kamala Harris, D-Calif.; Elizabeth Warren, D-Mass.; Cory Booker, D-N.J.; South Bend, Ind. Mayor Pete Buttigieg; former Rep. Beto O’Rourke, D-Texas, and former Housing and Urban Development Secretary Julian Castro all called for impeachment after the initial Times report.", "Harris continued to push for an investigation on Monday, even circulating an impeachment petition on her Twitter account."]
center_article = ["WASHINGTON – Rep. Ayanna Pressley, D-Mass., introduced an impeachment resolution against Supreme Court Justice Brett Kavanaugh after a new report of alleged sexual misconduct.", "“I believe Christine Blasey Ford. I believe Deborah Ramirez. It is our responsibility to collectively affirm the dignity and humanity of survivors,” she said in a statement to Boston public radio station WBUR.", "Pressley referred to the two women who accused Kavanaugh of sexual misconduct during his contentious nomination process last October.", "“Sexual predators do not deserve a seat on the nation’s highest court, and Brett Kavanaugh’s confirmation process set a dangerous precedent,” Pressley told WBUR. “We must demand justice for survivors and hold Kavanaugh accountable for his actions.”", "The two-page resolution asks the House Judiciary Committee to investigate the possibility of impeachment.", "Democratic leadership seems to have little appetite for a fight over impeachment, and the resolution is unlikely to go anywhere.", "Sen. Dick Durbin, D-Ill., the No. 2 Senate Democrat, told Politico on Monday, \"We’ve got to get beyond this ‘impeachment is the answer to every problem.’", "It’s not realistic.", "\"If that’s how we are identified in Congress, as the impeachment Congress, we run the risk that people will feel we’re ignoring the issues that mean a lot to them as families,\" he continued.", "An essay published in The New York Times on Sept. 14, adapted from a forthcoming book about Kavanaugh, included reporting about a newly surfaced misconduct allegation, as well as potential corroborating evidence for another allegation.", "The report sparked calls among Democrats for the justice's impeachment and investigation.", "The New York Times added an editor's note to the article about information on the allegation that Kavanaugh's \"friends pushed his penis into the hand of a female student\" that had been left out of the article originally. \"The book reports that the female student declined to be interviewed and friends say that she does not recall the incident,\" the Times said.", "Kavanaugh has denied the previously reported allegation and declined to be interviewed by the Times about the new allegation.", "Any attempt to impeach Kavanaugh is unlikely to succeed in the Republican-controlled Senate. Majority Leader Mitch McConnell, R-Ky., called impeachment a \"laughable suggestion\" Monday. Senate Judiciary Committee Chairman Lindsey Graham, R-S.C., vowed that Kavanaugh will not be impeached \"over these scurrilous accusations.\"", "Only once has a Supreme Court justice been impeached.", "Samuel Chase, a Federalist justice appointed by President George Washington, was impeached by a Democratic-Republican House in 1804 for \"arbitrary, oppressive, and unjust\" decisions on the court.", "The Senate declined to remove him from office."]
summary = generate_roundup_summary_from_text(event, "".join(left_article), "".join(center_article), "".join(right_article))

print("\n\n", summary)


Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.89s/it, est. speed input: 851.17 toks/s, output: 82.42 toks/s]



  Rep. Ayanna Pressley, a Democratic congresswoman, is pushing for an impeachment inquiry into Supreme Court Justice Brett Kavanaugh amidst new sexual misconduct allegations. She expressed her belief in the credibility of the accusers, Christine Blasey Ford and Deborah Ramirez, stating it is crucial to uphold the dignity and humanity of survivors. Pressley plans to introduce a resolution for the House Judiciary Committee to investigate the possibility of impeachment, despite hesitations from some top Democrats and doubts about the validity of the latest allegations. On the other hand, GOP lawmakers, including Senate Judiciary Committee Chairman Lindsey Graham, have vowed to defend Kavanaugh, with Graham denouncing the attacks as "beyond the pale." The New York Times initially reported a new allegation involving a dorm party incident, which was later corrected to state the alleged victim did not recall the sexual assault. Despite these developments, several Democratic presidential can

### Informational Bias Detection

In [15]:
import os
import torch
import argparse
import torch.nn as nn
from transformers import LongformerTokenizer, LongformerModel
from tqdm import tqdm
import numpy as np
import json

# Model: LongformerBiLSTM with Event Context
class LongformerBiLSTM(nn.Module):
    def __init__(self, hidden_dim=256, num_layers=2, num_classes=1, fine_tune_layers=2):
        super(LongformerBiLSTM, self).__init__()
        self.longformer = LongformerModel.from_pretrained("allenai/longformer-base-4096")

        # Freeze All Layers First
        for param in self.longformer.parameters():
            param.requires_grad = False

        # Unfreeze Last Few Layers
        for param in self.longformer.encoder.layer[-fine_tune_layers:].parameters():
            param.requires_grad = True

        self.lstm = nn.LSTM(input_size=self.longformer.config.hidden_size, hidden_size=hidden_dim,
                            num_layers=num_layers, batch_first=True, bidirectional=True)

        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim * 8, 256),  # Increased size for event-level embeddings
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes),
        )

    def encode_text(self, input_ids, attention_mask):
        outputs = self.longformer(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state
        _, (hidden, _) = self.lstm(last_hidden_state)
        embedding = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
        return embedding

    def forward(self, article_input_ids, article_attention_mask, sentence_input_ids, sentence_attention_mask,
                ev1_input_ids, ev1_attention_mask, ev2_input_ids, ev2_attention_mask):
        primary_embedding = self.encode_text(article_input_ids, article_attention_mask)
        sentence_embedding = self.encode_text(sentence_input_ids, sentence_attention_mask)
        ev1_embedding = self.encode_text(ev1_input_ids, ev1_attention_mask)
        ev2_embedding = self.encode_text(ev2_input_ids, ev2_attention_mask)

        combined_features = torch.cat([sentence_embedding, primary_embedding, ev1_embedding, ev2_embedding], dim=1)
        logits = self.classifier(combined_features).squeeze(1)
        return logits


def tokenize_batch(sentences, tokenizer, max_length):
    """Tokenize a batch of sentences"""
    return tokenizer(sentences, truncation=True, padding="max_length",
                    max_length=max_length, return_tensors="pt")

def run_inference(model, tokenizer, sentences, full_article, ev1_text, ev2_text,
                 device, batch_size=8, max_length=1024):
    model.eval()
    predictions = []

    article_encoding = tokenizer(full_article, truncation=True, padding="max_length",
                                max_length=max_length, return_tensors="pt")
    ev1_encoding = tokenizer(ev1_text, truncation=True, padding="max_length",
                            max_length=max_length, return_tensors="pt")
    ev2_encoding = tokenizer(ev2_text, truncation=True, padding="max_length",
                            max_length=max_length, return_tensors="pt")

    # Move these to device once
    article_input_ids = article_encoding["input_ids"].to(device)
    article_attention_mask = article_encoding["attention_mask"].to(device)
    ev1_input_ids = ev1_encoding["input_ids"].to(device)
    ev1_attention_mask = ev1_encoding["attention_mask"].to(device)
    ev2_input_ids = ev2_encoding["input_ids"].to(device)
    ev2_attention_mask = ev2_encoding["attention_mask"].to(device)

    # Process sentences in batches
    for i in range(0, len(sentences), batch_size):
        batch_sentences = sentences[i:i + batch_size]

        # Only tokenize the sentences for each batch
        sentence_encodings = tokenize_batch(batch_sentences, tokenizer, max_length)
        sentence_input_ids = sentence_encodings["input_ids"].to(device)
        sentence_attention_mask = sentence_encodings["attention_mask"].to(device)

        # Replicate the pre-tokenized article and event tensors for the batch size
        batch_size_actual = len(batch_sentences)
        batch_article_input_ids = article_input_ids.repeat(batch_size_actual, 1)
        batch_article_attention_mask = article_attention_mask.repeat(batch_size_actual, 1)
        batch_ev1_input_ids = ev1_input_ids.repeat(batch_size_actual, 1)
        batch_ev1_attention_mask = ev1_attention_mask.repeat(batch_size_actual, 1)
        batch_ev2_input_ids = ev2_input_ids.repeat(batch_size_actual, 1)
        batch_ev2_attention_mask = ev2_attention_mask.repeat(batch_size_actual, 1)

        with torch.no_grad():
            logits = model(
                batch_article_input_ids, batch_article_attention_mask,
                sentence_input_ids, sentence_attention_mask,
                batch_ev1_input_ids, batch_ev1_attention_mask,
                batch_ev2_input_ids, batch_ev2_attention_mask
            )

            # Convert to predictions (0 or 1)
            batch_preds = torch.sigmoid(logits).round().cpu().numpy().astype(int).tolist()
            predictions.extend(batch_preds)

    return predictions

def process_articles(model, tokenizer, left_sentences, right_sentences, center_sentences,
                    device, batch_size=8, max_length=1024):
    """Process pre-split sentences from left, right, and center articles"""
    results = {}

    # Create full text versions for context by joining sentences
    left_article = " ".join(left_sentences) if left_sentences else ""
    right_article = " ".join(right_sentences) if right_sentences else ""
    center_article = " ".join(center_sentences) if center_sentences else ""

    # Process left article sentences
    if left_sentences:
        left_preds = run_inference(
            model, tokenizer, left_sentences, left_article,
            right_article, center_article,
            device, batch_size, max_length
        )
        results["left"] = left_preds
    else:
        results["left"] = []

    # Process right article sentences
    if right_sentences:
        right_preds = run_inference(
            model, tokenizer, right_sentences, right_article,
            left_article, center_article,
            device, batch_size, max_length
        )
        results["right"] = right_preds
    else:
        results["right"] = []

    # Process center article sentences
    if center_sentences:
        center_preds = run_inference(
            model, tokenizer, center_sentences, center_article,
            left_article, right_article,
            device, batch_size, max_length
        )
        results["center"] = center_preds
    else:
        results["center"] = []

    return results


def detect_informational_bias_longformer_bilstm(left_article, right_article, center_article):
  checkpoint_path = "/content/drive/Shareddrives/FYP 2024-2025/Phase-2/Informational Bias Detection/models/model_ddp_epoch_10.pth"
  batch_size = 8
  max_length = 1024

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  # Load tokenizer
  tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")

  # Load model
  model = LongformerBiLSTM()
  checkpoint = torch.load(checkpoint_path, map_location=device)
  if "model_state_dict" in checkpoint:
      model.load_state_dict(checkpoint["model_state_dict"])
  else:
      model.load_state_dict(checkpoint)
  model.to(device)
  model.eval()

  # Process all article sentences
  results = process_articles(
      model, tokenizer,
      left_article, right_article, center_article,
      device, batch_size=batch_size, max_length=max_length
  )

  return results


In [16]:
# Test
left_article = ["WASHINGTON — House Judiciary Committee Chairman Jerry Nadler, D-N.Y., responded to calls for an investigation into Supreme Court Justice Brett Kavanaugh in light of new sexual misconduct allegations by saying that the panel has \"our hands full with impeaching the president.\"", "In a radio interview with WNYC on Monday, Nadler was asked if he’d be concerned with Democrats thinking he’s not taking the Kavanaugh allegations seriously enough.", "He said his committee has too much on its plate.", "\"We have our hands full with impeaching the president right now and that’s going to take up our limited resources and time for a while,\" Nadler said.", "Asked about whether he plans to keep his promise from last October of investigating Kavanaugh over his Senate confirmation hearing, Nadler was candid about his position on impeaching the president.", "\"Frankly, we are concentrating our resources on determining whether to impeach the president. Personally I think the President ought to be impeached but we have to concentrate on that for the next few months,\" he said.", "Over the weekend, a pair of New York Times reporters in an opinion-section article previewing their upcoming book on Kavanaugh's confirmation said they found new corroboration for accusations that Kavanaugh exposed himself to Deborah Ramirez, a classmate at Yale.", "NBC News has not verified that reporting.", "The reporters said that former Yale classmate Max Stier told senators and the FBI about a different episode of alleged sexual misconduct.", "Two people with first-hand knowledge confirm to NBC News that the FBI was notified of Stier's claim.", "The chairman said that his committee plans to question FBI Director Christopher Wray next month including about the agency’s investigation into Kavanaugh.", "Nadler continued by saying he personally believes impeaching the president is \"imperative\" so that Democrats can \"vindicate the Constitution,\" acknowledging that its purpose would not be removing Trump from office because the GOP-controlled Senate wouldn’t do that.", "Nadler said that while he feels there is already enough information to impeach the President, he believes the Committee needs to educate the public to get behind impeachment before moving forward.", "\"We have to show that there are adequate grounds for impeachment, that there are imperative grounds for impeachment and convince people. If that happens, if people are convinced after the hearings that the president should be impeached then we will be able to get the votes, if that doesn’t happen, we won’t be able to get the votes,\" Nadler said.", "The Judiciary Committee approved a resolution last Thursday that set procedures and rules for future impeachment investigation hearings.", "The first such hearing will be held on Tuesday afternoon with Trump’s former campaign manager, Corey Lewandowski, as the first witness.", "Some top Democrats, including Sens. Kamala Harris, D-Calif. and Elizabeth Warren, D-Mass., called for Kavanaugh to be impeached in light of the new allegations against him.", "The revelations today confirm what we already knew: During his hearing, Kavanaugh faced credible accusations and likely lied to Congress. I support any appropriate constitutional mechanism to hold him accountable. — Bernie Sanders (@BernieSanders) September 15, 2019", "Kavanaugh was sworn in to the high court last October after senators investigated sexual assault and misconduct allegations made against him by Christine Blasey Ford and other women during his Senate confirmation process.", "He has denied the allegations made against him."]
right_article = ["Rep. Ayanna Pressley, D-Mass., one of the four freshman congresswomen known collectively as the “Squad,” is planning Tuesday to introduce a resolution seeking an impeachment inquiry into Supreme Court Associate Justice Brett Kavanaugh, despite reluctance from top Democrats to pursue one and concerns about the basis for the latest accusations.", "The resolution follows a New York Times report that included a new allegation of sexual misconduct from when Kavanaugh was in college -- although an editors' note later revealed that the alleged victim did not recall it taking place.", "In a statement, Pressley only focused on the women who came forward during Kavanaugh’s confirmation process.", "“I believe Christine Blasey Ford. I believe Deborah Ramirez,” Pressley said in a statement shared by Boston.com reporter Nik DeCosta-Klipa, which Pressley retweeted from her official Twitter account. “It is our responsibility to collectively affirm the dignity and humanity of survivors.”", "Pressley also used her personal Twitter account to retweet a post that included images of what appears to be text from the resolution.", "It calls for “Inquiring whether the House of Representatives should impeach Brett Michael Kavanaugh, Associate Justice for the Supreme Court of the United States of America,” and for the House Judiciary Committee to conduct depositions and take affidavits, serving subpoenas if necessary.", "Rep. Jerrold Nadler, D-N.Y., who heads the House Judiciary Committee, has not necessarily come out against impeaching Kavanaugh.", "But he made clear it was not a top priority for his committee.", "“We have our hands full with impeaching the president right now,” Nadler said on a radio show Monday.", "Top-ranking Sen. Dick Durbin, D-Ill., outright rejected the calls.", "“We’ve got to get beyond this ‘impeachment is the answer to every problem,” Durbin said, according to Politico. “It’s not realistic. If that’s how we are identified in Congress, as the impeachment Congress, we run the risk that people will feel we’re ignoring the issues that mean a lot to them as families.”", "GOP lawmakers, meanwhile, quickly rallied around Kavanaugh Monday and vowed to fight any impeachment effort.", "South Carolina Republican Sen. Lindsey Graham, the current Judiciary Committee chairman who mounted one of the most passionate defenses of Kavanaugh during his confirmation hearing, called the attacks “beyond the pale.”", "“My heart goes out to Justice Kavanaugh’s family for being forced to endure this ridiculous treatment once again,” he tweeted.", "The original Times piece, adapted from a forthcoming book, asserted that a Kavanaugh classmate, Clinton-connected nonprofit CEO Max Stier, “saw Mr. Kavanaugh with his pants down at a different drunken dorm party, where friends pushed his penis into the hand of a female student.”", "By late Sunday, the paper had revised its story to include the fact that several friends of the alleged victim said she told them she did not recall the reported sexual assault in question.", "Several Democratic presidential candidates, however, said that Kavanaugh should be impeached and did not back down after the editors' note and revision to the story.", "Sens. Kamala Harris, D-Calif.; Elizabeth Warren, D-Mass.; Cory Booker, D-N.J.; South Bend, Ind. Mayor Pete Buttigieg; former Rep. Beto O’Rourke, D-Texas, and former Housing and Urban Development Secretary Julian Castro all called for impeachment after the initial Times report.", "Harris continued to push for an investigation on Monday, even circulating an impeachment petition on her Twitter account."]
center_article = ["WASHINGTON – Rep. Ayanna Pressley, D-Mass., introduced an impeachment resolution against Supreme Court Justice Brett Kavanaugh after a new report of alleged sexual misconduct.", "“I believe Christine Blasey Ford. I believe Deborah Ramirez. It is our responsibility to collectively affirm the dignity and humanity of survivors,” she said in a statement to Boston public radio station WBUR.", "Pressley referred to the two women who accused Kavanaugh of sexual misconduct during his contentious nomination process last October.", "“Sexual predators do not deserve a seat on the nation’s highest court, and Brett Kavanaugh’s confirmation process set a dangerous precedent,” Pressley told WBUR. “We must demand justice for survivors and hold Kavanaugh accountable for his actions.”", "The two-page resolution asks the House Judiciary Committee to investigate the possibility of impeachment.", "Democratic leadership seems to have little appetite for a fight over impeachment, and the resolution is unlikely to go anywhere.", "Sen. Dick Durbin, D-Ill., the No. 2 Senate Democrat, told Politico on Monday, \"We’ve got to get beyond this ‘impeachment is the answer to every problem.’", "It’s not realistic.", "\"If that’s how we are identified in Congress, as the impeachment Congress, we run the risk that people will feel we’re ignoring the issues that mean a lot to them as families,\" he continued.", "An essay published in The New York Times on Sept. 14, adapted from a forthcoming book about Kavanaugh, included reporting about a newly surfaced misconduct allegation, as well as potential corroborating evidence for another allegation.", "The report sparked calls among Democrats for the justice's impeachment and investigation.", "The New York Times added an editor's note to the article about information on the allegation that Kavanaugh's \"friends pushed his penis into the hand of a female student\" that had been left out of the article originally. \"The book reports that the female student declined to be interviewed and friends say that she does not recall the incident,\" the Times said.", "Kavanaugh has denied the previously reported allegation and declined to be interviewed by the Times about the new allegation.", "Any attempt to impeach Kavanaugh is unlikely to succeed in the Republican-controlled Senate. Majority Leader Mitch McConnell, R-Ky., called impeachment a \"laughable suggestion\" Monday. Senate Judiciary Committee Chairman Lindsey Graham, R-S.C., vowed that Kavanaugh will not be impeached \"over these scurrilous accusations.\"", "Only once has a Supreme Court justice been impeached.", "Samuel Chase, a Federalist justice appointed by President George Washington, was impeached by a Democratic-Republican House in 1804 for \"arbitrary, oppressive, and unjust\" decisions on the court.", "The Senate declined to remove him from office."]
results = detect_informational_bias_longformer_bilstm(left_article, right_article, center_article)
print("Results:", results)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/597M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/597M [00:00<?, ?B/s]

Results: {'left': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], 'right': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0], 'center': [0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0]}


### Flask app

In [17]:
import getpass
import os
import threading

from flask import Flask,request,jsonify
from flask_cors import CORS
from pyngrok import ngrok, conf

print("Enter your ngrok authtoken: ")
conf.get_default().auth_token = getpass.getpass()

app = Flask(__name__)
CORS(app)
port = "5000"

public_url = ngrok.connect(port).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"")

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# Define Flask routes
@app.route("/")
def index():
    return "Hello World!"

@app.route("/api/lexical_detection", methods=['POST'])
def lexical_detection():
  try:
      # Get the input data (list of sentences)
      data = request.json
      sentences = data.get("sentences", [])
      if not isinstance(sentences, list) or not all(isinstance(s, str) for s in sentences):
          return jsonify({"error": "Invalid input. Expecting a JSON object with a 'sentences' list."}), 400
      # Run detection for each sentence
      results = [run_detection(sentence) for sentence in sentences]
      # Return the list of predictions
      return jsonify({"predictions": results})
  except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route("/api/lexical_mitigation", methods = ['POST'])
def lexical_mitigation():
  try:
      # Get the input data (biased sentence)
      data = request.json
      sentence = data.get("sentence", "")
      if not isinstance(sentence, str):
          return jsonify({"error": "Invalid input. Expecting a JSON object with a 'sentence' string."}), 400
      # Run mitigation for the sentence
      debiased_sentence = run_mitigation(sentence)
      # Return the debiased sentence
      return jsonify({"debiased_sentence": debiased_sentence})
  except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route("/api/informational_detection", methods = ['POST'])
def informational_detection():
  try:
    data = request.json
    left_article = data.get("left_article", [])
    right_article = data.get("right_article", [])
    center_article = data.get("center_article", [])
    info_bias = detect_informational_bias_longformer_bilstm(left_article, right_article, center_article)
    return info_bias
  except Exception as e:
    return jsonify({"error": str(e)}), 500

@app.route("/api/roundup", methods = ['POST'])
def roundup_without_perspectives():
  try:
    data = request.json
    event_headline = data.get("event_headline", "")
    left_article = data.get("left_article", [])
    right_article = data.get("right_article", [])
    center_article = data.get("center_article", [])
    left_article = ' '.join(left_article)
    right_article = ' '.join(right_article)
    center_article = ' '.join(center_article)
    roundup = generate_roundup_summary_from_text(event_headline, left_article,center_article, right_article)
    return jsonify({"roundup": roundup})
  except Exception as e:
    return jsonify({"error": str(e)}), 500


# Run Flask app in a thread
def run_app():
    app.run(host="0.0.0.0", port=5000, debug=False)  # Disable debug mode

# Start Flask in a separate thread
thread = threading.Thread(target=run_app)
thread.start()

Enter your ngrok authtoken: 
··········
 * ngrok tunnel "https://e296-34-87-156-136.ngrok-free.app" -> "http://127.0.0.1:5000"


### Test Flask App Inference

In [ ]:
import requests

In [18]:
url ="https://e296-34-87-156-136.ngrok-free.app"

In [19]:
# Lexical Bias Detection
det_url = url+"/api/lexical_detection"
data = {
    "sentences": [
        "Our new marketing strategy triggered a nuclear explosion of sales, completely overshadowing all competitors.",
        "Critics deemed the policy drastically misguided, but supporters applauded it as boldly innovative and a step toward needed reform.", #Synonyms and Paraphrases
        "While Senator Goodwin meticulously outlined a data-driven agenda, Senator Hasting spoke in broad generalities, prompting speculation that Hasting lacks the necessary expertise to handle the issue effectively." #Subtle Framing Devices

    ]
}

try:
    response = requests.post(det_url, json=data)
    print("Status Code:", response.status_code)
    print("Raw Response:", response.text)  # Print the raw response
    if response.status_code == 200:
        print("Predictions:", response.json()["predictions"])
    else:
        print("Error:", response.json())
except requests.exceptions.RequestException as e:
    print("Request failed:", e)
except ValueError as e:
    print("Response parsing failed:", e)

INFO:werkzeug:127.0.0.1 - - [05/Apr/2025 08:15:43] "POST /api/lexical_detection HTTP/1.1" 200 -


Status Code: 200
Raw Response: {"predictions":[1,0,1]}

Predictions: [1, 0, 1]


In [20]:
# Lexical Bias Mitigation
mit_url = url+"/api/lexical_mitigation"
data = {
    "sentence": "People from that region are lazy, slow, and completely ignorant of modern society"
}

try:
    response = requests.post(mit_url, json=data)
    print("Status Code:", response.status_code)
    print("Raw Response:", response.text)  # Print the raw response
    if response.status_code == 200:
        print("Predictions:", response.json()["debiased_sentence"])
    else:
        print("Error:", response.json())
except requests.exceptions.RequestException as e:
    print("Request failed:", e)
except ValueError as e:
    print("Response parsing failed:", e)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
INFO:werkzeug:127.0.0.1 - - [05/Apr/2025 08:15:48] "POST /api/lexical_mitigation HTTP/1.1" 200 -


Status Code: 200
Raw Response: {"debiased_sentence":"People from that region are perceived as resistant to change and less familiar with modern society."}

Predictions: People from that region are perceived as resistant to change and less familiar with modern society.


In [21]:
event_headline = "Pressley To Push For Kavanaugh Impeachment Inquiry"
left_article = ["WASHINGTON — House Judiciary Committee Chairman Jerry Nadler, D-N.Y., responded to calls for an investigation into Supreme Court Justice Brett Kavanaugh in light of new sexual misconduct allegations by saying that the panel has \"our hands full with impeaching the president.\"", "In a radio interview with WNYC on Monday, Nadler was asked if he’d be concerned with Democrats thinking he’s not taking the Kavanaugh allegations seriously enough.", "He said his committee has too much on its plate.", "\"We have our hands full with impeaching the president right now and that’s going to take up our limited resources and time for a while,\" Nadler said.", "Asked about whether he plans to keep his promise from last October of investigating Kavanaugh over his Senate confirmation hearing, Nadler was candid about his position on impeaching the president.", "\"Frankly, we are concentrating our resources on determining whether to impeach the president. Personally I think the President ought to be impeached but we have to concentrate on that for the next few months,\" he said.", "Over the weekend, a pair of New York Times reporters in an opinion-section article previewing their upcoming book on Kavanaugh's confirmation said they found new corroboration for accusations that Kavanaugh exposed himself to Deborah Ramirez, a classmate at Yale.", "NBC News has not verified that reporting.", "The reporters said that former Yale classmate Max Stier told senators and the FBI about a different episode of alleged sexual misconduct.", "Two people with first-hand knowledge confirm to NBC News that the FBI was notified of Stier's claim.", "The chairman said that his committee plans to question FBI Director Christopher Wray next month including about the agency’s investigation into Kavanaugh.", "Nadler continued by saying he personally believes impeaching the president is \"imperative\" so that Democrats can \"vindicate the Constitution,\" acknowledging that its purpose would not be removing Trump from office because the GOP-controlled Senate wouldn’t do that.", "Nadler said that while he feels there is already enough information to impeach the President, he believes the Committee needs to educate the public to get behind impeachment before moving forward.", "\"We have to show that there are adequate grounds for impeachment, that there are imperative grounds for impeachment and convince people. If that happens, if people are convinced after the hearings that the president should be impeached then we will be able to get the votes, if that doesn’t happen, we won’t be able to get the votes,\" Nadler said.", "The Judiciary Committee approved a resolution last Thursday that set procedures and rules for future impeachment investigation hearings.", "The first such hearing will be held on Tuesday afternoon with Trump’s former campaign manager, Corey Lewandowski, as the first witness.", "Some top Democrats, including Sens. Kamala Harris, D-Calif. and Elizabeth Warren, D-Mass., called for Kavanaugh to be impeached in light of the new allegations against him.", "The revelations today confirm what we already knew: During his hearing, Kavanaugh faced credible accusations and likely lied to Congress. I support any appropriate constitutional mechanism to hold him accountable. — Bernie Sanders (@BernieSanders) September 15, 2019", "Kavanaugh was sworn in to the high court last October after senators investigated sexual assault and misconduct allegations made against him by Christine Blasey Ford and other women during his Senate confirmation process.", "He has denied the allegations made against him."]
right_article = ["Rep. Ayanna Pressley, D-Mass., one of the four freshman congresswomen known collectively as the “Squad,” is planning Tuesday to introduce a resolution seeking an impeachment inquiry into Supreme Court Associate Justice Brett Kavanaugh, despite reluctance from top Democrats to pursue one and concerns about the basis for the latest accusations.", "The resolution follows a New York Times report that included a new allegation of sexual misconduct from when Kavanaugh was in college -- although an editors' note later revealed that the alleged victim did not recall it taking place.", "In a statement, Pressley only focused on the women who came forward during Kavanaugh’s confirmation process.", "“I believe Christine Blasey Ford. I believe Deborah Ramirez,” Pressley said in a statement shared by Boston.com reporter Nik DeCosta-Klipa, which Pressley retweeted from her official Twitter account. “It is our responsibility to collectively affirm the dignity and humanity of survivors.”", "Pressley also used her personal Twitter account to retweet a post that included images of what appears to be text from the resolution.", "It calls for “Inquiring whether the House of Representatives should impeach Brett Michael Kavanaugh, Associate Justice for the Supreme Court of the United States of America,” and for the House Judiciary Committee to conduct depositions and take affidavits, serving subpoenas if necessary.", "Rep. Jerrold Nadler, D-N.Y., who heads the House Judiciary Committee, has not necessarily come out against impeaching Kavanaugh.", "But he made clear it was not a top priority for his committee.", "“We have our hands full with impeaching the president right now,” Nadler said on a radio show Monday.", "Top-ranking Sen. Dick Durbin, D-Ill., outright rejected the calls.", "“We’ve got to get beyond this ‘impeachment is the answer to every problem,” Durbin said, according to Politico. “It’s not realistic. If that’s how we are identified in Congress, as the impeachment Congress, we run the risk that people will feel we’re ignoring the issues that mean a lot to them as families.”", "GOP lawmakers, meanwhile, quickly rallied around Kavanaugh Monday and vowed to fight any impeachment effort.", "South Carolina Republican Sen. Lindsey Graham, the current Judiciary Committee chairman who mounted one of the most passionate defenses of Kavanaugh during his confirmation hearing, called the attacks “beyond the pale.”", "“My heart goes out to Justice Kavanaugh’s family for being forced to endure this ridiculous treatment once again,” he tweeted.", "The original Times piece, adapted from a forthcoming book, asserted that a Kavanaugh classmate, Clinton-connected nonprofit CEO Max Stier, “saw Mr. Kavanaugh with his pants down at a different drunken dorm party, where friends pushed his penis into the hand of a female student.”", "By late Sunday, the paper had revised its story to include the fact that several friends of the alleged victim said she told them she did not recall the reported sexual assault in question.", "Several Democratic presidential candidates, however, said that Kavanaugh should be impeached and did not back down after the editors' note and revision to the story.", "Sens. Kamala Harris, D-Calif.; Elizabeth Warren, D-Mass.; Cory Booker, D-N.J.; South Bend, Ind. Mayor Pete Buttigieg; former Rep. Beto O’Rourke, D-Texas, and former Housing and Urban Development Secretary Julian Castro all called for impeachment after the initial Times report.", "Harris continued to push for an investigation on Monday, even circulating an impeachment petition on her Twitter account."]
center_article = ["WASHINGTON – Rep. Ayanna Pressley, D-Mass., introduced an impeachment resolution against Supreme Court Justice Brett Kavanaugh after a new report of alleged sexual misconduct.", "“I believe Christine Blasey Ford. I believe Deborah Ramirez. It is our responsibility to collectively affirm the dignity and humanity of survivors,” she said in a statement to Boston public radio station WBUR.", "Pressley referred to the two women who accused Kavanaugh of sexual misconduct during his contentious nomination process last October.", "“Sexual predators do not deserve a seat on the nation’s highest court, and Brett Kavanaugh’s confirmation process set a dangerous precedent,” Pressley told WBUR. “We must demand justice for survivors and hold Kavanaugh accountable for his actions.”", "The two-page resolution asks the House Judiciary Committee to investigate the possibility of impeachment.", "Democratic leadership seems to have little appetite for a fight over impeachment, and the resolution is unlikely to go anywhere.", "Sen. Dick Durbin, D-Ill., the No. 2 Senate Democrat, told Politico on Monday, \"We’ve got to get beyond this ‘impeachment is the answer to every problem.’", "It’s not realistic.", "\"If that’s how we are identified in Congress, as the impeachment Congress, we run the risk that people will feel we’re ignoring the issues that mean a lot to them as families,\" he continued.", "An essay published in The New York Times on Sept. 14, adapted from a forthcoming book about Kavanaugh, included reporting about a newly surfaced misconduct allegation, as well as potential corroborating evidence for another allegation.", "The report sparked calls among Democrats for the justice's impeachment and investigation.", "The New York Times added an editor's note to the article about information on the allegation that Kavanaugh's \"friends pushed his penis into the hand of a female student\" that had been left out of the article originally. \"The book reports that the female student declined to be interviewed and friends say that she does not recall the incident,\" the Times said.", "Kavanaugh has denied the previously reported allegation and declined to be interviewed by the Times about the new allegation.", "Any attempt to impeach Kavanaugh is unlikely to succeed in the Republican-controlled Senate. Majority Leader Mitch McConnell, R-Ky., called impeachment a \"laughable suggestion\" Monday. Senate Judiciary Committee Chairman Lindsey Graham, R-S.C., vowed that Kavanaugh will not be impeached \"over these scurrilous accusations.\"", "Only once has a Supreme Court justice been impeached.", "Samuel Chase, a Federalist justice appointed by President George Washington, was impeached by a Democratic-Republican House in 1804 for \"arbitrary, oppressive, and unjust\" decisions on the court.", "The Senate declined to remove him from office."]

In [23]:
# Informational Bias Detection
info_bias_url = url+"/api/informational_detection"
data = {
    "left_article": left_article,
    "right_article": right_article,
    "center_article": center_article
}

try:
    response = requests.post(info_bias_url, json=data)
    print("Status Code:", response.status_code)
    print("Raw Response:", response.text)  # Print the raw response
except requests.exceptions.RequestException as e:
    print("Request failed:", e)
except ValueError as e:
    print("Response parsing failed:", e)


INFO:werkzeug:127.0.0.1 - - [05/Apr/2025 08:16:22] "POST /api/informational_detection HTTP/1.1" 200 -


Status Code: 200
Raw Response: {"center":[0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,1,0],"left":[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0],"right":[0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0]}



In [25]:
# Neutral Roundup
roundup_url = url+"/api/roundup"
data = {
    "event_headline": event_headline,
    "left_article": left_article,
    "right_article": right_article,
    "center_article": center_article
}

try:
    response = requests.post(roundup_url, json=data)
    print("Status Code:", response.status_code)
    print("Raw Response:", response.text)
    if response.status_code == 200:
        print("Roundup:", response.json()["roundup"])
    else:
        print("Error:", response.json())
except requests.exceptions.RequestException as e:
    print("Request failed:", e)
except ValueError as e:
    print("Response parsing failed:", e)

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.38s/it, est. speed input: 726.19 toks/s, output: 67.47 toks/s]
INFO:werkzeug:127.0.0.1 - - [05/Apr/2025 08:17:15] "POST /api/roundup HTTP/1.1" 200 -


Status Code: 200
Raw Response: {"roundup":" The event revolves around Rep. Ayanna Pressley, a Democrat, introducing a resolution to impeach Supreme Court Justice Brett Kavanaugh due to allegations of sexual misconduct surfaced during his Senate confirmation and later in college. The left-leaning perspective, represented by the left article, focuses on Pressley's call for an impeachment inquiry, emphasizing her belief in the credibility of the women who came forward during Kavanaugh's confirmation process. The center article, representing a moderate perspective, highlights the reluctance of Democratic leadership to pursue impeachment, with Sen. Dick Durbin voicing concerns about it becoming a distraction from other issues. The right article, from a conservative viewpoint, underscores the Republican GOP's swift defense of Kavanaugh, with Sen. Lindsey Graham labeling the attacks as \"beyond the pale.\" All perspectives agree on the existence of allegations, but diverge on the need for imp